<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 2.1 MB/s 


In [9]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [10]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [11]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE',
       'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD',
       'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI', 'MCDOWELL-N',
       'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [12]:
mypf['InPortfolio'].value_counts()

0    36
1    24
Name: InPortfolio, dtype: int64

In [13]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

946.0

In [14]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [15]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [16]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-10-10 00:00:00+05:30,17254.0,17319.0,17274.0,47.568547,17109.0,^NSEI,0.85,18477.0,15294.0,61.59,38.41


In [17]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [18]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-10-10 00:00:00+05:30,702.0,740.0,758.0,28.240643,816.0,MOTILALOFS.NS,-13.96,990.0,702.0,0.03,99.97,1000.0,42.45
1,2022-10-10 00:00:00+05:30,396.0,418.0,433.0,34.953145,441.0,TATAMOTORS.NS,-10.17,530.0,372.0,15.28,84.72,530.0,33.84
2,2022-10-10 00:00:00+05:30,534.0,539.0,549.0,44.921603,580.0,HDFCLIFE.NS,-8.02,721.0,504.0,13.59,86.41,690.0,29.21
3,2022-10-10 00:00:00+05:30,3802.0,3982.0,4070.0,39.533968,4486.0,NAUKRI.NS,-15.25,7020.0,3375.0,11.71,88.29,4867.0,28.01
5,2022-10-10 00:00:00+05:30,72.0,73.0,76.0,45.748877,88.0,NATIONALUM.NS,-17.90,130.0,67.0,8.33,91.67,91.0,26.39
6,2022-10-10 00:00:00+05:30,2301.0,2346.0,2357.0,43.022652,2397.0,HDFC.NS,-3.99,3001.0,2053.0,26.20,73.80,2905.0,26.25
7,2022-10-10 00:00:00+05:30,1417.0,1446.0,1447.0,42.495451,1452.0,HDFCBANK.NS,-2.41,1689.0,1281.0,33.32,66.68,1780.0,25.62
8,2022-10-10 00:00:00+05:30,76.0,78.0,78.0,46.153969,95.0,IBREALEST.NS,-20.16,191.0,60.0,12.10,87.90,95.0,25.00
10,2022-10-10 00:00:00+05:30,1051.0,1037.0,1048.0,53.216158,1186.0,MUTHOOTFIN.NS,-11.36,1713.0,956.0,12.59,87.41,1279.0,21.69
11,2022-10-10 00:00:00+05:30,778.0,757.0,748.0,56.872725,732.0,AXISBANK.NS,6.22,845.0,626.0,69.20,30.80,946.0,21.59


In [19]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     9.710626
1     7.741050
2     6.681917
3     6.407412
5     6.036829
6     6.004804
7     5.860689
8     5.718861
10    4.961684
11    4.938808
12    4.886195
14    4.703191
15    4.652865
16    4.616264
17    4.389798
22    3.380991
24    3.104198
29    2.646689
31    2.582638
32    2.511724
35    2.225781
42    1.040833
52    0.226467
59   -5.030310
Name: Upside%, dtype: float64

In [20]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-10-10 00:00:00+05:30,516.0,540.0,550.0,32.243226,552.0,ICICIPRULI.NS,-6.56,674.0,447.0,30.31,69.69,654.0,26.74
9,2022-10-10 00:00:00+05:30,6150.0,6332.0,6372.0,37.364512,6507.0,ULTRACEMCO.NS,-5.49,8214.0,5177.0,32.04,67.96,7570.0,23.09
13,2022-10-10 00:00:00+05:30,1241.0,1263.0,1256.0,42.511018,1194.0,SBILIFE.NS,3.97,1332.0,1025.0,70.50,29.50,1505.0,21.27
18,2022-10-10 00:00:00+05:30,1029.0,1039.0,1055.0,45.563441,1198.0,TECHM.NS,-14.07,1806.0,965.0,7.66,92.34,1226.0,19.14
19,2022-10-10 00:00:00+05:30,529.0,558.0,561.0,32.716456,557.0,DABUR.NS,-5.00,624.0,485.0,31.76,68.24,625.0,18.15
20,2022-10-10 00:00:00+05:30,1779.0,1853.0,1876.0,36.911641,1890.0,INDIGO.NS,-5.88,2364.0,1549.0,28.21,71.79,2081.0,16.98
21,2022-10-10 00:00:00+05:30,2570.0,2672.0,2725.0,36.989069,2681.0,HEROMOTOCO.NS,-4.14,2928.0,2199.0,50.91,49.09,2988.0,16.26
23,2022-10-10 00:00:00+05:30,758.0,794.0,795.0,36.980797,779.0,TATACONSUM.NS,-2.71,853.0,660.0,50.73,49.27,863.0,13.85
25,2022-10-10 00:00:00+05:30,1812.0,1844.0,1849.0,43.143811,1844.0,KOTAKBANK.NS,-1.71,2211.0,1642.0,29.95,70.05,2055.0,13.41
26,2022-10-10 00:00:00+05:30,411.0,405.0,413.0,54.406808,487.0,WIPRO.NS,-15.66,722.0,393.0,5.40,94.60,466.0,13.38


In [21]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
14,2022-10-10 00:00:00+05:30,1464.0,1432.0,1464.0,55.074395,1568.0,INFY.NS,-6.64,1940.0,1365.0,17.21,82.79,1765.0,20.56
18,2022-10-10 00:00:00+05:30,1029.0,1039.0,1055.0,45.563441,1198.0,TECHM.NS,-14.07,1806.0,965.0,7.66,92.34,1226.0,19.14
22,2022-10-10 00:00:00+05:30,961.0,934.0,941.0,61.565386,1039.0,HCLTECH.NS,-7.52,1352.0,883.0,16.60,83.40,1103.0,14.78
26,2022-10-10 00:00:00+05:30,411.0,405.0,413.0,54.406808,487.0,WIPRO.NS,-15.66,722.0,393.0,5.40,94.60,466.0,13.38
29,2022-10-10 00:00:00+05:30,3119.0,3075.0,3143.0,52.973272,3360.0,TCS.NS,-7.19,4019.0,2982.0,13.17,86.83,3480.0,11.57
31,2022-10-10 00:00:00+05:30,3695.0,3683.0,3688.0,52.199471,4024.0,DIVISLAB.NS,-8.19,5372.0,3449.0,12.77,87.23,4112.0,11.29
37,2022-10-10 00:00:00+05:30,2608.0,2631.0,2590.0,47.576707,2448.0,HINDUNILVR.NS,6.55,2702.0,1944.0,87.63,12.37,2784.0,6.75
38,2022-10-10 00:00:00+05:30,328.0,331.0,321.0,48.848195,282.0,ITC.NS,16.37,346.0,208.0,87.07,12.93,350.0,6.71
43,2022-10-10 00:00:00+05:30,623.0,631.0,637.0,43.183037,669.0,BERGEPAINT.NS,-6.83,838.0,561.0,22.49,77.51,651.0,4.49
46,2022-10-10 00:00:00+05:30,617.0,617.0,602.0,51.028049,609.0,JUBLFOOD.NS,1.25,905.0,464.0,34.60,65.40,634.0,2.76
